In [15]:
import numpy as np
import tensorflow as tf
import keras

from importlib import reload
import src.text_generation as text_gen


ModuleNotFoundError: No module named 'src'

In [ ]:

path_to_file = keras.utils.get_file(
        'shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
        )
print(path_to_file)

with open(path_to_file) as f:
    shakespeare_text = f.read()
print(shakespeare_text[:148])

shakespeare_tensor = tf.constant([shakespeare_text])
vocab = list(set(shakespeare_text.lower().strip()))
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts(shakespeare_text.lower())
n_tokens = len(tokenizer.word_index)

/tmp/.keras/datasets/shakespeare.txt
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



2025-05-08 18:36:36.848565: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-05-08 18:36:36.851904: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
from keras.models import load_model

# Load the model from the saved path
model = load_model("models/shakespeare_gru_1.keras")
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, None, 128)         64896     
                                                                 
 dense_1 (Dense)             (None, None, 39)          5031      
                                                                 
Total params: 69,927
Trainable params: 69,927
Non-trainable params: 0
_________________________________________________________________


2025-05-08 18:42:54.472952: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-05-08 18:42:54.473841: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-05-08 18:42:54.474447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
text = "to be "
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
print(sequence)
print(tokenizer.sequences_to_texts([sequence]))
sequence = sequence - 1
print(sequence)

[ 3  4  1 22  2  1]
['t o   b e  ']
[ 2  3  0 21  1  0]


In [ ]:
inp = tf.one_hot(sequence, depth=n_tokens)
print(inp.shape)
inp_tf = tf.reshape(inp, (1, 6, n_tokens))
print(inp_tf.shape)

(6, 39)
(1, 6, 39)


In [ ]:
y = model.predict(inp_tf)
print(y.shape)
prediction = y[0, -1, :]
print(prediction.shape)

1/1 [==============================] - 0s 179ms/step
(1, 6, 39)
(39,)


2025-05-08 18:44:04.592222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-05-08 18:44:04.593007: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-05-08 18:44:04.593546: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
char_id = np.argmax(prediction) + 1
print(char_id)

sequence = np.append(sequence + 1, char_id)
print(sequence)
tokenizer.sequences_to_texts([sequence])

5
[ 4  5  2 23  3  2  6  5]


['o a e p t e i a']

In [ ]:
reload(text_gen)
one_hot_fn = lambda x: tf.one_hot(x, depth=n_tokens)
sequence = np.array(tokenizer.texts_to_sequences([text])[0])
sequence = sequence - 1
text_gen.next_token(sequence, model, transform=one_hot_fn, argmax=False, temperature=10)

NameError: name 'text_gen' is not defined

In [ ]:
from src.text_generation import next_token

next_token(sequence, model, transform=one_hot_fn, argmax=False, temperature=10)

ModuleNotFoundError: No module named 'src'

In [ ]:
reload(text_gen)

text_gen.generate("to be ", model, tokenizer, n_chars=50, temperature=0.3, argmax=True)